In [2]:
!pip install faiss-cpu opencv-python tensorflow scikit-learn matplotlib pillow ipywidgets

print("Block 1: No error")

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/15.0 MB 5.6 MB/s eta 0:00:03
   ------------- -------------------------- 5.2/15.0 MB 16.0 MB/s eta 0:00:01
   ----------------------------- ---------- 11.0/15.0 MB 20.2 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 20.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 30.7 MB/s eta 0:00:00

   -------- ------------------------------- 1/5 [jupyterlab_widgets]
   ---------------- ----------------------- 2/5 [faiss-cpu]
   ---------------- ----------------------- 2/5 [faiss-cpu]
   ---------------- ----------------------- 2/5 [faiss-cpu]
   ---------------- ----------------------- 2/5 [faiss-cpu]
   ---------------- ----------------------- 2/5 [faiss-cpu]
   ---------------- ---

In [4]:
import os
import cv2
import numpy as np
import time
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing import image

from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, clear_output

# Configure for speed
tf.config.experimental.enable_memory_growth = True
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print("Block 2: No error")


Block 2: No error


In [5]:
class FastConfig:
    embedding_dim = 128
    image_size = (160, 160)
    top_k = 6
    cache_dir = 'fast_cache'

config = FastConfig()

print("Block 3: No error")

Block 3: No error


In [6]:
class UltraFastExtractor:
    def __init__(self):
        self.base_model = MobileNetV2(
            weights='imagenet', 
            include_top=False, 
            pooling='avg',
            input_shape=(*config.image_size, 3)
        )
        
        self.model = tf.keras.Sequential([
            self.base_model,
            tf.keras.layers.Dense(config.embedding_dim, activation='relu'),
            tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))
        ])
        
        self.model.compile(optimizer='adam')
        dummy_input = np.random.random((1, *config.image_size, 3)).astype(np.float32)
        self.model.predict(dummy_input, verbose=0)
    
    def extract_fast(self, img_array):
        if len(img_array.shape) == 3:
            img_array = np.expand_dims(img_array, axis=0)
        
        img_array = preprocess_input(img_array.astype(np.float32))
        embedding = self.model.predict(img_array, verbose=0)
        return embedding.flatten()

extractor = UltraFastExtractor()

print("Block 4: No error")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

Block 4: No error


In [7]:
class LightningSearch:
    def __init__(self):
        self.indices = {}
        self.paths = {}
        self.thumbnails = {}
        
    def load_precomputed_data(self, gender):
        cache_file = f"{config.cache_dir}/{gender}_fast_cache.npz"
        
        if not os.path.exists(cache_file):
            print(f"Cache file not found: {cache_file}")
            return False
        
        data = np.load(cache_file, allow_pickle=True)
        embeddings = data['embeddings']
        paths = data['paths']
        thumbnails = data['thumbnails']
        
        index = faiss.IndexFlatIP(config.embedding_dim)
        index.add(embeddings.astype('float32'))
        
        self.indices[gender] = index
        self.paths[gender] = paths
        self.thumbnails[gender] = thumbnails
        
        return True
    
    def search_lightning_fast(self, query_embedding, gender, k=6):
        if gender not in self.indices:
            if not self.load_precomputed_data(gender):
                return [], []
        
        query_norm = query_embedding / np.linalg.norm(query_embedding)
        query_norm = query_norm.reshape(1, -1).astype('float32')
        
        similarities, indices = self.indices[gender].search(query_norm, k)
        
        result_thumbnails = [self.thumbnails[gender][idx] for idx in indices[0]]
        scores = similarities[0]
        
        return result_thumbnails, scores

search_engine = LightningSearch()

print("Block 5: No error")

Block 5: No error


In [8]:
def capture_face_ultra_fast():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    print("Press ENTER to capture, ESC to exit")
    
    captured_face = None
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 'Face Detected!', (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.putText(frame, 'ENTER: Capture | ESC: Exit', (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Face Capture', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == 13:  # Enter
            if len(faces) > 0:
                x, y, w, h = max(faces, key=lambda f: f[2]*f[3])
                face_img = frame[y:y+h, x:x+w]
                captured_face = cv2.resize(face_img, config.image_size)
                break
            else:
                print("No face detected!")
        elif key == 27:  # Escape
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return captured_face

print("Block 6: No error")

Block 6: No error


In [10]:
def create_instant_collage(thumbnails, scores):
    cols, rows = 3, 2
    thumb_size = 200
    margin = 20
    
    collage_width = cols * thumb_size + (cols + 1) * margin
    collage_height = rows * thumb_size + (rows + 1) * margin + 60
    
    collage = Image.new('RGB', (collage_width, collage_height), (40, 40, 40))
    
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        font = ImageFont.load_default()
    
    draw = ImageDraw.Draw(collage)
    title = "Your Top Matches"
    title_bbox = draw.textbbox((0, 0), title, font=font)
    title_width = title_bbox[2] - title_bbox[0]
    title_x = (collage_width - title_width) // 2
    draw.text((title_x, 10), title, fill=(255, 255, 255), font=font)
    
    for i, (thumbnail, score) in enumerate(zip(thumbnails, scores)):
        if i >= 6:
            break
            
        row = i // cols
        col = i % cols
        
        x = margin + col * (thumb_size + margin)
        y = 50 + margin + row * (thumb_size + margin)
        
        if isinstance(thumbnail, np.ndarray):
            if thumbnail.shape[2] == 3:
                thumbnail = cv2.cvtColor(thumbnail, cv2.COLOR_BGR2RGB)
            pil_thumb = Image.fromarray(thumbnail)
        else:
            pil_thumb = thumbnail
            
        pil_thumb = pil_thumb.resize((thumb_size, thumb_size), Image.Resampling.LANCZOS)
        collage.paste(pil_thumb, (x, y))
        
        score_text = f"{score:.3f}"
        score_bbox = draw.textbbox((0, 0), score_text, font=font)
        score_bg = Image.new('RGBA', (score_bbox[2] + 10, score_bbox[3] + 4), (0, 0, 0, 180))
        collage.paste(score_bg, (x + 5, y + thumb_size - 25), score_bg)
        draw.text((x + 10, y + thumb_size - 23), score_text, fill=(255, 255, 255), font=font)
    
    return collage

print("Block 7: No error")

Block 7: No error


In [11]:
def preprocess_dataset_for_speed(dataset_dir, gender):
    os.makedirs(config.cache_dir, exist_ok=True)
    
    gender_dir = Path(dataset_dir) / gender / gender
    if not gender_dir.exists():
        print(f"Dataset directory not found: {gender_dir}")
        return
    
    image_paths = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
        image_paths.extend(gender_dir.glob(ext))
    
    if not image_paths:
        print(f"No images found in {gender_dir}")
        return
    
    print(f"Found {len(image_paths)} images")
    
    embeddings = []
    thumbnails = []
    valid_paths = []
    
    batch_size = 32
    total_processed = 0
    
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch_images = []
        batch_thumbnails = []
        batch_valid_paths = []
        
        for path in batch_paths:
            try:
                img = cv2.imread(str(path))
                if img is None:
                    continue
                    
                img_resized = cv2.resize(img, config.image_size)
                batch_images.append(img_resized)
                
                thumbnail = cv2.resize(img, (200, 200))
                batch_thumbnails.append(thumbnail)
                
                batch_valid_paths.append(str(path))
                
            except Exception as e:
                continue
        
        if not batch_images:
            continue
        
        batch_embeddings = []
        for img in batch_images:
            try:
                embedding = extractor.extract_fast(img)
                batch_embeddings.append(embedding)
            except:
                continue
        
        embeddings.extend(batch_embeddings)
        thumbnails.extend(batch_thumbnails)
        valid_paths.extend(batch_valid_paths)
        
        total_processed += len(batch_embeddings)
        print(f"Processed {total_processed}/{len(image_paths)} images")
    
    if not embeddings:
        print(f"No valid embeddings extracted for {gender}")
        return
    
    embeddings = np.array(embeddings)
    thumbnails = np.array(thumbnails)
    valid_paths = np.array(valid_paths)
    
    cache_file = f"{config.cache_dir}/{gender}_fast_cache.npz"
    np.savez_compressed(
        cache_file,
        embeddings=embeddings,
        paths=valid_paths,
        thumbnails=thumbnails
    )
    
    print(f"Preprocessing complete for {gender}")
    print(f"Cache saved: {cache_file}")

print("Block 8: No error")

Block 8: No error


In [12]:
def get_instant_recommendations():
    gender = input("Enter gender (male/female): ").strip().lower()
    
    if gender not in ['male', 'female']:
        print("Invalid gender! Please enter 'male' or 'female'")
        return
    
    print(f"Starting recommendations for {gender}...")
    
    face_img = capture_face_ultra_fast()
    
    if face_img is None:
        print("No face captured!")
        return
    
    print("Analyzing your face...")
    face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    embedding = extractor.extract_fast(face_rgb)
    
    print("Finding matches...")
    thumbnails, scores = search_engine.search_lightning_fast(embedding, gender, k=6)
    
    if not thumbnails:
        print(f"No matches found for gender: {gender}")
        return
    
    print("Creating collage...")
    collage = create_instant_collage(thumbnails, scores)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(collage)
    plt.axis('off')
    plt.title('Your Face Recommendations', fontsize=16, pad=20)
    plt.tight_layout()
    plt.show()
    
    print("Recommendations complete!")

print("Block 9: No error")

Block 9: No error


In [13]:
get_instant_recommendations()

Enter gender (male/female):  male


Starting recommendations for male...
Press ENTER to capture, ESC to exit
Analyzing your face...
Finding matches...
Cache file not found: fast_cache/male_fast_cache.npz
No matches found for gender: male
